In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Objective :
## Given a review findout that review is positive or negative

### How should we determine review is positive or negative?
    
>  We will use score as our matrix for defining the polarity of reviews
 if rating is 4 or 5 we will consider it positive
 if rating is 2 or 1 we will consider it negative
 if rating is 3 we will ignore as it is neutral.
 

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# for more information about  Python regular expressions visit :  https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

ModuleNotFoundError: No module named 'gensim'

1.  ** Reading data**

In [ ]:
# for kaggle
# conn = sqlite3.connect("/kaggle/input/amazon-fine-food-reviews/database.sqlite")
conn = sqlite3.connect("./database.sqlite")

filtered_input = pd.read_sql_query("""
                                    SELECT * FROM 
                                     Reviews
                                   WHERE SCORE != 3
                                  """, conn)                                                                
filtered_input.head()

In [ ]:
#  change the reviews >  3 positve rating and < 3 negative rating

def rating_parser(score):
    if score < 3:
        return 0
    return 1

#  now update the score rating in the score column
filtered_input["Score"].value_counts()
true_score = filtered_input["Score"]

# true_score.value_counts()
updated_score = true_score.map(rating_parser)
# #  update the score now
updated_score.value_counts()
filtered_input["Score"] =  updated_score



In [ ]:
print("no of data in the input",filtered_input.shape)
filtered_input.head()

# Data cleaning and Data Processing

In [ ]:
#  lets review some duplicated reviews in dataset
display = pd.read_sql_query("""
    SELECT UserId,ProductId, ProfileName,Time,Score, Text,Count(*)
    FROM Reviews
    GROUP BY UserId
    having Count(*) > 1
    """, conn
)
display.head(100)

In [79]:
# in dataset we have some user id and product id combination is same but timestamp and text of the review is different
#  this should be handled accordingly your need
g = pd.read_sql_query( """
select UserId,ProductId,count(1) from Reviews group by UserId,ProductId
having count(1) >1
                     """,conn)

g.head(100)
# lets check one of thosed data
filtered_input[ (filtered_input["UserId"] == "A102TGNH1D915Z" ) & (filtered_input["ProductId"] == "B00008DFK5")]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
220784,239325,B00008DFK5,A102TGNH1D915Z,LINDA Riley,2,3,1,1262131200,Not just for puppies...,"Although the product can be a bit pricey, it c..."
220793,239337,B00008DFK5,A102TGNH1D915Z,LINDA Riley,0,0,1,1264204800,Great distraction,When I want my dog to leave everyone alone for...


# Exploratory Data Analysis
## Data DeDuplication

### Data Might contains lots of duplicate like because of data collection error it 
### is necessary to remove those duplicated for unbiased analysis

In [82]:
# we can cleary see that below record is duplicated because the text and time stamp of the reviews are same which is very unlikely for the same 
# user
filtered_input[filtered_input["UserId"] == "#oc-R11D9D7SHXIJB9"]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
268373,290948,B005HG9ESG,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",0,0,1,1342396800,Muscle spasms,"My wife has recurring extreme muscle spasms, u..."
421207,455534,B005HG9ERW,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",0,0,1,1342396800,Muscle spasms,"My wife has recurring extreme muscle spasms, u..."
459536,496894,B005HG9ET0,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",0,0,1,1342396800,Muscle spasms,"My wife has recurring extreme muscle spasms, u..."


In [85]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="#oc-R11D9D7SHXIJB9"
ORDER BY ProductID
""", conn)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,455534,B005HG9ERW,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",0,0,5,1342396800,Muscle spasms,"My wife has recurring extreme muscle spasms, u..."
1,290948,B005HG9ESG,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",0,0,5,1342396800,Muscle spasms,"My wife has recurring extreme muscle spasms, u..."
2,496894,B005HG9ET0,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",0,0,5,1342396800,Muscle spasms,"My wife has recurring extreme muscle spasms, u..."


> # as we can see above the same user has multiple reviews with same time summary text profilename, helpfullnessNumerator and helpfullnessdenominator, only different thing is the productid
 

 Doing further analysis I found below information 

 detail page of  B005HG9ESG
 https://www.amazon.com/dp/B005HG9ESG

 detail page of B005HG9ERW
 https://www.amazon.com/dp/B005HG9ERW

 from above detail page of product we can infer that both the product are same and different only in quantity and flavour
 that is why this review is appearing in both the products
 so here this is not usefull  information ebcause it is redudant so we will only keep the first such review after sorting all the reviews by product id


In [86]:
sorted_data = filtered_input.sort_values?

In [95]:
# sort the filterd data frame by product Id 
sorted_data = filtered_input.sort_values("ProductId", axis=0, na_position="last",ascending=True, inplace=False, kind="quicksort")
print(sorted_data.shape)
#  remove all the duplicated and keep first record
final = sorted_data.drop_duplicates(subset={"UserId","ProductId","Time","Text"}, keep="first",inplace=False)
print(final.shape)


(525814, 10)
(524691, 10)


In [98]:
# checking % of data remains after removal
(final['Id'].size)/(filtered_input['Id'].size)*100

99.78642637890965

In [100]:
#  in the data set  HelpfulnessNumerator should be less than HelpfulnessDenominator
# This two record should be removed from the data
filtered_input_1 = filtered_input[filtered_input["HelpfulnessDenominator"] < filtered_input["HelpfulnessNumerator"]]
filtered_input_1

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
41159,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,1,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...
59301,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,1,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...


In [101]:
#  so we will filter out above rows
final =  final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]
final.shape

(524689, 10)

In [105]:
final.shape
# let's find out how many positive and negative ratings are there in our dataset
final["Score"].value_counts()

1    442966
0     81723
Name: Score, dtype: int64

Id                        524691
ProductId                 524691
UserId                    524691
ProfileName               524691
HelpfulnessNumerator      524691
HelpfulnessDenominator    524691
Score                     524691
Time                      524691
Summary                   524691
Text                      524691
dtype: int64

In [ ]:
final = sorted_data.drop_duplicates

In [ ]:
d

In [ ]:
display['Count(*)'].sum()